### 1- Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt

### 2- Fouille de données

In [ ]:
FILE = '../data/raw/providers.csv'
df = pd.read_csv(FILE)

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.isna().any()

In [ ]:
sorted(df.company_id.unique()), df.company_id.nunique()

In [ ]:
df_companies = df.query('provider_id.isnull()')[['company_id', 'fullname']]
print (df_companies)

In [ ]:
df_groupby_count_providers = df.groupby('company_id')['provider_id'].nunique()
df_to_plot = df_companies.join (df_groupby_count_providers, on=['company_id'])
ax = df_to_plot.plot(kind='barh'
                    , x='fullname'
                    , y='provider_id'
                    , figsize= (10,10))

ax.set_xlabel('Nombres de transports')
ax.set_title('répartition des sous- compagnies en fonction des compagnies')
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
df_groupby_transport_type = df.groupby('transport_type')['company_id'].nunique()
ax = df_groupby_transport_type.plot(kind='pie'
                                    , figsize= (10,10)
                                    , title='Distribution du type de service'
                                    , ylabel=''
                                    , xlabel=''
                                    , autopct='%.2f'
                                    )
plt.show()

In [ ]:
df_groupby_count_transport_type = df.groupby('transport_type')['transport_type'].count()

ax = df_groupby_count_transport_type.plot.barh(figsize = (10,10))

ax.set_xlabel('Nombres de transports')
ax.set_title('Nombres de transports en fonction du type')
ax.bar_label(ax.containers[0])
plt.show()


In [ ]:
df_copy = df.copy()
dict_to_replace = {}
for i in df_companies['company_id']:
    dict_to_replace[i] = df_companies.loc[df_companies['company_id'] == i].fullname
df_copy = df_copy.replace ({'company_id':dict_to_replace})

In [ ]:
df_groupby_count_company_transport = df_copy.groupby(['company_id', 'transport_type'] )['transport_type'].count()

In [ ]:
ax = df_groupby_count_company_transport.unstack().plot(kind='bar'
                                                        , layout=(4, 1)
                                                        , figsize=(10,10)
                                                        , subplots=True
                                                        , title='Répartition des types de transport en fonction des compagnies'
                                                        , rot=30)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)
                        , nrows=2
                        , ncols=2)
axe = ax.ravel()
titles = ['Has wifi ?', 'Has Plug ?', 'has adjustable seats ?', 'Has bicycle ?']
for i, column in enumerate(['has_wifi', 'has_plug', 'has_adjustable_seats', 'has_bicycle']):
    df_copy[column].value_counts().plot(kind='pie'
                                        , title=titles[i]
                                        , ylabel=''
                                        , xlabel=''
                                        , autopct='%.2f'
                                        , ax=axe[i])
plt.show()


In [ ]:
df_duplicated = df[df.duplicated(["fullname","has_wifi","has_plug","has_adjustable_seats","has_bicycle","transport_type"], keep=False)]
print (df_duplicated)

### Analyses

- id : identifiant
- company_id : identifiant de la compagnie
- provider_id : identifiant de la sous compagnie
- name : nom
- fullname : nom officiel 
- has_wifi : wifi présent ou non
- has_plug : prise présente ou non
- has_adjustable_seats : siège ajustable ou non
- has_bicycle : porte vélo ou non 
- transport_type : type de transport

Lorsque une ligne contient un company_id et pas de provider_id, il s'agit de la compagnie qui englobe des sous-compagnies. <br/>
Il y a 14 compagnies : <br/>
[Ouibus, Deinbus.de, Infobus, Student Agency, Flixbus, DriiveMe, Alsa, Distribusion, Voyages SNCF, Transdev, Blablacar, Vatry, Beauval, idVroom] <br/>
Distribusion détient plus de 90% des sous compagnies. Le reste est partagé avec Voyage SNCF et Transdev. <br/>
Près de 94% des transports sont des bus. <br/>
73% des compagnies proposent des bus comme moyen de locomotion. <br/>
On remarque des monopoles : bus (Distribusion), car (DriiveMe) et train (Voyages SNCF) <br/>
Le carpooling est partagé entre IdVroom et Blablacar.

Parmi les providers, on remarque la présence de 2 doublons : TER et Intercités. D'après le nom donné à ces doublons, il se peut que certaines données proviennent de différentes sources, ce qui expliquerait la multiplication de noms pour un même provider. <br/>

### Data Preparation


In [ ]:
df_copy = df_copy.drop_duplicates(subset=["fullname","has_wifi","has_plug","has_adjustable_seats","has_bicycle","transport_type"])

In [ ]:
df_copy.to_csv('../data/cleaned/providers_cleaned.csv')